# Local level model

This note uses a local level model. Basic state-space model with measurement error and an AR(1) latent process.

$$y_t = \gamma + \eta z_t + \epsilon_t, \quad \epsilon_t \sim N_{0,\sigma^2}, \quad t=1,\dots,\tau$$
$$z_t = \rho z_{t-1} + \nu_t\quad \nu_t \sim N_{0,1}, \quad t=2,\dots,\tau$$
$$z_1 \sim N_{0, (1+\rho^2)^{-1/2}} $$

See the graphs below and code for information about the priors and inference, which is a structured (full-rank) VI procedure.

## Preliminaries

In [1]:
# !pip3 install http://download.pytorch.org/whl/cu90/torch-0.4.0-cp36-cp36m-linux_x86_64.whl >/dev/null && echo OK
# !pip3 install torchvision > /dev/null && echo OK
# !pip3 uninstall -y ptvi >/dev/null && echo OK
# !pip3 install git+https://github.com/kuperov/ptvi.git  >/dev/null && echo OK

In [2]:
from ptvi import *
import torch
import pandas as pd
pd.set_option('precision', 2)

Define the true parameter values and generate data.

In [3]:
γ0, η0, σ0, ρ0 = 0., 2., 1.5, 0.92

In [4]:
torch.manual_seed(1234)
y, z = LocalLevelModel(τ=100).simulate(γ=γ0, η=η0, σ=σ0, ρ=ρ0)

## Approximate inference

Our inference procedure is quick, but frankly not super great.

There are two different ways to compute elbo estimates. The first is ADVI, where our elbo is estimated with an analytic entropy term, as

$$ \hat{\mathcal{L}} = \frac{1}{M} \sum_{i=1}^M 
    p(y, \zeta^{(i)}) - 
    \mathbb{E}_{q_{u,L}} \left[ q_{u,L}(\zeta^{(i)}) 
   \right]
$$
where $\zeta^{(i)}\sim q_{u,L}$ via the reparameterization trick. 

The other is SGVI, where the elbo is estimated by simulating the entropy term, as

$$ \hat{\mathcal{L}} = \frac{1}{M} \sum_{i=1}^M \left[
    p(y, \zeta^{(i)}) - q_{u,L}(\zeta^{(i)}) 
   \right]
$$
where $\zeta^{(i)}\sim q_{u,L}$ via the reparameterization trick. 

The code is available [here](https://github.com/kuperov/ptvi/blob/dev/ptvi/local_level.py).

In [ ]:
model = LocalLevelModel(τ=100, stoch_entropy=True, num_draws=10)
fit = model.training_loop(y, max_iters=2**20)

Local level model with τ=100:
    - stochastic entropy;
    - 10 simulation draws;
    - Exponential stopping heuristic (N=50, M=50, α=0.05)

RMSprop optimizer with param groups:
    group 0. lr=0.01, momentum=0, alpha=0.99, eps=1e-08, centered=False, weight_decay=0

Displayed loss is smoothed with λ=0.1
       0. smoothed elbo_hat =-2002.72
       1. smoothed elbo_hat = 1621.74
       2. smoothed elbo_hat =-1691.85
       4. smoothed elbo_hat = -283.80
       8. smoothed elbo_hat =-3997.65
      16. smoothed elbo_hat =-3633.93
      32. smoothed elbo_hat = -476.42
      64. smoothed elbo_hat = 1019.79
     128. smoothed elbo_hat = -236.61
     256. smoothed elbo_hat =   86.89
     512. smoothed elbo_hat =  -57.30
    1024. smoothed elbo_hat =  -15.60
    2048. smoothed elbo_hat =  -13.86
    4096. smoothed elbo_hat =  -13.51
    8192. smoothed elbo_hat =  -14.21


In [ ]:
fit.plot_elbos()

In [ ]:
fit.summary()

Note the model's overconfidence about the latent state. You can barely even see the standard deviation band around the blue estimated state.

In [ ]:
fit.plot_latent(true_z=z.numpy(), include_data=True)

In [ ]:
plot_dens({'p(σ | y)': fit.σ_marg_post, 'p(σ)': model.σ_prior, '$σ_0$': σ0}, -1, 10)

In [ ]:
plot_dens({'p(γ | y)': fit.γ_marg_post, 'p(γ)': model.γ_prior, '$γ_0$': γ0}, -4, 4)

In [ ]:
plot_dens({'p(η | y)': fit.η_marg_post, 'p(η)': model.η_prior, '$η_0$': η0}, -1, 10)

In [ ]:
plot_dens({'p(ρ | y)': fit.ρ_marg_post, 'p(ρ)': model.ρ_prior, '$ρ_0$': ρ0}, -1.5, 1.5)

### Forecasts

In [ ]:
fit.plot_sampled_paths(N=100, true_y=y, fc_steps=20)

In [ ]:
fit.plot_pred_ci(N=500, α=.1, true_y=y, fc_steps=20)